<a href="https://colab.research.google.com/github/sergiogf93/MetNumerics/blob/master/notebooks/09_ODE_BVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

%matplotlib inline
import numpy
import matplotlib.pyplot as plt

# Resoldre problemes de condicions de frontera

Volem resoldre una EDO la qual té condicions de frontera en comptes de condicions de valor inicial. Aquest és un problema que apareix en problemes espacials on l'EDO és continguda en un interval i volem imposar valors als extrems. Un dels casos més simples és el problema de Poisson unidimensional

$$
    u_{xx} = f(x)
$$
on fem servir la notació
$$
    u_{xx} = \frac{\text{d}^2 u}{\text{d} x^2} \quad \text{or} \quad \frac{\partial^2 u}{\partial x^2}.
$$

Noteu que degut a l'ordre de la derivada es requereixen dues condicions per resoldre el problema. El cas més simple quan tenim el domini $x \in [a,b]$ és tenir condicions del tipus $u(a) = u_a$ i $u(b) = u_b$ i s'anomenen problemes de condicions de frontera (PCF). 

Si les condicions són ambdues en un extrem del domini podem dir que tenim de nou un problema de valor inicial. Així doncs, què necessitem per resoldre aquest tipus de problemes? Considerarem dues tècniques:

1. Reformular el problema de condicions de frontera com un problema de valors inicial i utilitzem els mètodes per resoldre EDOs.
2. Utilitzem diferències finites per representar les incògnites con un sistema lineal que resolem.

## El mètode de tir

El mètode de tir reformula el problema com un problema de càlcul d'arrels. Vegem-ne un exemple.

Considerem el problema
$$
    u_{xx} = -\sin u
$$
on
$$
    x \in [0, 2] \quad \text{i} \quad u(0) = 0.0, \quad u(2.0) = \frac{\pi}{2}.
$$

Podem reescriure el problema com un sistema de dues EDO
$$
    v = \begin{bmatrix} u \\ u_x \end{bmatrix} \quad \text{i} \quad v_x = \begin{bmatrix} u_x \\ u_{xx} \end{bmatrix} = \begin{bmatrix} v_2 \\ -\sin v_1 \end{bmatrix}.
$$

Sabem que volem $v_1(0)=0$ però què utilitzem per $v_2(0)$? Si fem una suposició inicial per $v_2(0)$ i resolem l'EDO de valors inicial associada podem trobar quina de les d'aquestes condicions inicials ens dóna la condició de contorn a la dreta. Utilitzant tècniques de trobar arrels (o trobar mínims) podem escriure aquest procediment com

$$
    \min_{v_2(0)} \left | \pi / 2 - v_1(2) \right |
$$

on el paràmetre que variem és $v_2(0)$.

In [0]:
# Basic Shooting Method solving u_xx = -sin(u)
import scipy.integrate as integrate

# Algorithm parameters
TOLERANCE = 1e-8
MAX_ITERATIONS = 100

# Problem Statement
a = 0.0
b = 2.0
N = 100
x = numpy.linspace(a, b, N)
u_a = 0.0
u_b = numpy.pi / 2.0
# RHS function
def f(x, u):
    return numpy.array([u[1], -numpy.sin(u[0])])

# Initial guess
# Slope at RHS
u_prime_rhs = 1.0
# Initial step size
du_prime = 0.5

# Plotting
fig = plt.figure()
fig.set_figwidth(fig.get_figwidth() * 2)
axes = fig.add_subplot(1, 2, 1)

# Main loop
success = False
u = numpy.empty((2, N))
convergence = numpy.zeros(MAX_ITERATIONS)
for n in range(MAX_ITERATIONS):
    
    # Initial condition
    u[0, 0] = u_a
    u[1, 0] = u_prime_rhs

    # Construct integrator
    integrator = integrate.ode(f)
    integrator.set_integrator("dopri5")
    integrator.set_initial_value(u[:, 0])

    # Compute solution - note that we are only producing the intermediate values
    # for demonstration purposes
    for (i, x_output) in enumerate(x[1:]):
        integrator.integrate(x_output)
        if not integrator.successful():
            raise Exception("Integration Failed!")
        u[:, i + 1] = integrator.y

    # Stopping Criteria
    convergence[n] = numpy.abs(u[0, -1] - u_b)
    if numpy.abs(u[0, -1] - u_b) < TOLERANCE:
        success = True
        break
    else:
        if u[0, -1] < u_b:
            u_prime_rhs += du_prime
        else:
            u_prime_rhs -= du_prime
            du_prime *= 0.5

    axes.plot(x, u[0, :], 'b')
    axes.plot(b, u_b, 'ro')

axes.set_title("Shooting Method Iterations")
axes.set_xlabel("$x$")
axes.set_ylabel("$u(x)$")

axes = fig.add_subplot(1, 2, 2)
n_range = numpy.arange(n)
axes.semilogy(n_range, convergence[:n])
axes.set_title("Convergence of Shooting Method")
axes.set_xlabel("step")
axes.set_ylabel("$|u(b) - U(b)|$")

plt.show()

La part complicada d'aquest mètode és decidir el criteri de cerca, és a dir, com canviar $v_2(0)$ respecte de la posició $v_1(2)$ comparat amb què volem $u(2)$.


En general qualsevol algoritme de minimització es pot fer servir en el mètode tir. Aquestes tècniques són molt efectives en general per aproximar PCFs no-lineals. El problema serà si són massa exigents computacionalment.

## Sistemes lineals

### Formulació

La segona tècnica a considerar consisteix a formar un sistema d'equacions a resoldre basat en aproximacions en diferències finites. Com abans, considerem el problema on
$$
    u_{xx} = f(x)
$$

amb les condicions $u(a) = u_a$ i $u(b) = u_b$.  

Ja sabem que la diferència finita de segon ordre per aproximar la segona derivada d'una funció $u(x)$ és

$$
    u_{xx} \approx \frac{u(x_{i-1}) - 2 u(x_i) + u(x_{i+1})}{\Delta x^2}.
$$

Si discretitzem el domini del problema original en $N$ punts (sense incloure els extrems) tal que

$$
    x_i = a + \frac{b - a}{N+1} \cdot i ~~~ \text{on} ~~~ i = 1, \ldots, N
$$

podem escriure l'aproximació en diferències finites com un sistema lineal d'equacions!

Si prenem per exemple $N = 5$ llavors

$$\begin{aligned}
    (U_{xx})_1 &\approx \frac{U_a - 2 U_1 + U_2}{\Delta x^2} \\
    (U_{xx})_2 &\approx \frac{U_1 - 2 U_2 + U_3}{\Delta x^2} \\
    (U_{xx})_3 &\approx \frac{U_2 - 2 U_3 + U_4}{\Delta x^2} \\
    (U_{xx})_4 &\approx \frac{U_3 - 2 U_4 + U_5}{\Delta x^2} \\
    (U_{xx})_5 &\approx \frac{U_4 - 2 U_5 + U_b}{\Delta x^2} \\
\end{aligned}$$

on hem utilitzat que $U_a = u(a)$ i $U_b = u(b)$ són les condicions de contorn.

A partir d'aquestes aproximacions de les derivades podem escriure l'EDO com

$$
    \frac{1}{\Delta x^2}\begin{bmatrix}
    -2 &  1 &    &    &    \\
     1 & -2 &  1 &    &    \\
       &  1 & -2 &  1 &    \\
       &    &  1 & -2 &  1 \\
       &    &    &  1 & -2 \\
    \end{bmatrix} \begin{bmatrix}
        U_1 \\ U_2 \\ U_3 \\ U_4 \\ U_5
    \end{bmatrix} = 
    \begin{bmatrix}
        f(x_1) \\ f(x_2) \\ f(x_3) \\ f(x_4) \\ f(x_5) \\
    \end{bmatrix}.
$$

Tingueu en compte que el nostre exemple anterior utilitzat per al mètode de tir és difícil en el context actual ja que la funció desconeguda està en la funció $ f $ de manera que necessitem solucionar un sistema no lineal d'equacions. Això encara seria possible si utilitzem altres mètodes per trobar la solució però l'implementació es comença a complicar.

### Condicions de frontera

Això no inclou les condicions de frontera però. Podem afegir aquests valors fàcilment per condicions de frontera de Dirichlet passant els valors coneguts al vector $b$:

$$\begin{aligned}
    \frac{U_a - 2 U_1 + U_2}{\Delta x^2} = f(x_1) &\Rightarrow& \frac{- 2 U_1 + U_2}{\Delta x^2} = f(x_1) - \frac{U_a}{\Delta x^2} \\
    \frac{U_4 - 2 U_5 + U_b}{\Delta x^2} = f(x_1) &\Rightarrow& \frac{U_4 - 2 U_5}{\Delta x^2} = f(x_5) - \frac{U_b}{\Delta x^2}
\end{aligned}$$
així doncs el sistema final queda
$$
    \frac{1}{\Delta x^2} \begin{bmatrix}
    -2 &  1 &    &    &    \\
     1 & -2 &  1 &    &    \\
       &  1 & -2 &  1 &    \\
       &    &  1 & -2 &  1 \\
       &    &    &  1 & -2 \\
    \end{bmatrix} \begin{bmatrix}
        U_1 \\ U_2 \\ U_3 \\ U_4 \\ U_5
    \end{bmatrix} = 
    \begin{bmatrix}
        f(x_1) - \frac{U_a}{\Delta x^2} \\ f(x_2) \\ f(x_3) \\ f(x_4) \\ f(x_5) - \frac{U_b}{\Delta x^2} \\
    \end{bmatrix}.
$$

### Exemple

Volem resoldre el PCF
$$
    u_{xx} = e^x, \quad x \in [0, 1] \quad \text{amb} \quad u(0) = 0.0, \text{ i } u(1) = 3
$$
mitjançant la construcció d'un sistema linear d'equacions.


\begin{align*}
    u_{xx} &= e^x \\
    u_x &= A + e^x \\
    u &= Ax + B + e^x\\
    u(0) &= B + 1 = 0 \Rightarrow B = -1 \\
    u(1) &= A - 1 + e^{1} = 3 \Rightarrow A = 4 - e\\ 
    ~\\
    u(x) &= (4 - e) x - 1 + e^x
\end{align*}

In [0]:
# Problem setup
a = 0.0
b = 1.0
u_a = 0.0
u_b = 3.0
f = lambda x: numpy.exp(x)
u_true = lambda x: (4.0 - numpy.exp(1.0)) * x - 1.0 + numpy.exp(x)

# Descretization
N = 10
x_bc = numpy.linspace(a, b, N + 2)
x = x_bc[1:-1]
delta_x = (b - a) / (N + 1)

# Construct matrix A
A = numpy.zeros((N, N))
diagonal = numpy.ones(N) / delta_x**2
A += numpy.diag(diagonal * -2.0, 0)
A += numpy.diag(diagonal[:-1], 1)
A += numpy.diag(diagonal[:-1], -1)

# Construct RHS
b = f(x)
b[0] -= u_a / delta_x**2
b[-1] -= u_b / delta_x**2

# Solve system
U = numpy.empty(N + 2)
U[0] = u_a
U[-1] = u_b
U[1:-1] = numpy.linalg.solve(A, b)

# Plot result
fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(x_bc, U, 'o', label="Computed")
axes.plot(x_bc, u_true(x_bc), 'k', label="True")
axes.set_title("Solution to $u_{xx} = e^x$")
axes.set_xlabel("x")
axes.set_ylabel("u(x)")
plt.show()

Si tenim condicions de contorn de Neumann ja no és tant clar com incorporar-les. Normalment es fa servir un **ghost cell** que afegeix les incògnites que representen els valors de contorn que coneixem.

Per exemple, si tenim el PCF

$$
    u_{xx} = e^x, \quad x \in [-1, 1] \quad \text{amb} \quad u(-1) = 3, \text{ i } u_x(1) = -5
$$
llavors podem tenir les condicions de frontera al vector d'incògnites de manera que
$$
    U = \begin{bmatrix} U_0 \\ U_1 \\ \vdots \\ U_N \\ U_{N+1} \end{bmatrix}
$$
on aquí $U_0$ i $U_{N+1}$ són de fet les condicions de frontera.

La matriu $A$ és llavors modificada de manera que contingui les relacions adequades. En el cas de la condició de frontera de l'esquerra tenim

$$
    A = \begin{bmatrix}
  1 &    &    &    &    &    \\
  \frac{1}{\Delta x^2} & \frac{-2}{\Delta x^2} &  \frac{1}{\Delta x^2} &    &    &    \\
    &  \frac{1}{\Delta x^2} & \frac{-2}{\Delta x^2} &  \frac{1}{\Delta x^2} &    &    \\
    & & \ddots & \ddots & \ddots
    \end{bmatrix} \quad \text{and} \quad b = \begin{bmatrix}
        u(a) \\ f(x_1) \\ f(x_2) \\ \vdots
    \end{bmatrix}
$$
que donaria

$$
    U_0 = u(-1) = 3.
$$

Per la condició de la dreta podem utilitzar la diferència finita de segon ordre com aproximació de la primera derivada

$$
    u_x(b) \approx \frac{3 U_{N+1} - 4 U_{N} + U_{N - 1}}{2.0 \Delta x} = -5
$$
que incorporades a la matriu $A$ i al vector $b$ queda
$$
    A =  \begin{bmatrix}
     \ddots & \ddots & \ddots &    &    \\
            & \frac{1}{\Delta x^2} &     \frac{-2}{\Delta x^2}&  \frac{1}{\Delta x^2} &    \\
            &        &      \frac{1}{\Delta x^2} & \frac{-2}{\Delta x^2} &  \frac{1}{\Delta x^2} \\
            &        &      \frac{1}{2 \Delta x} &  \frac{-4}{2 \Delta x} &  \frac{3}{2 \Delta x} \\
    \end{bmatrix} ~~~~ \text{i} ~~~~ b = \begin{bmatrix}
        \vdots \\ f(x_N) \\ u_x(b)
    \end{bmatrix}.
$$

Tot junt el now sistema és

$$
    \begin{bmatrix}
     1 &    &    &    &    &    \\
     \frac{1}{\Delta x^2} & \frac{-2}{\Delta x^2} &  \frac{1}{\Delta x^2} &    &    &    \\
       &  \ddots & \ddots &  \ddots &    \\
       &    & \frac{1}{\Delta x^2} & \frac{-2}{\Delta x^2} &  \frac{1}{\Delta x^2} \\
            &        &      \frac{1}{2 \Delta x} &  \frac{-4}{2 \Delta x} &  \frac{3}{2 \Delta x} \\
    \end{bmatrix} \begin{bmatrix}
        U_0 \\ U_1 \\ \vdots \\ U_N \\ U_{N+1}
    \end{bmatrix} = 
    \begin{bmatrix}
        u(a) \\ f(x_1) \\ \vdots \\ f(x_N) \\ u_x(b)
    \end{bmatrix}.
$$

### Exercici

Volem resoldre el PCF

$$
    u_{xx} = e^x, \quad x \in [-1, 1] \quad \text{amb} \quad u(-1) = 3.0, \text{ i } u_x(1) = -5.0
$$
a partir de la construcció d'un sistema linear d'equacions.

Troba primer la solució real i després implementa el codi per comparar-la amb el resultat obtingut amb el mètode del sistema d'equacions.

#### Solució real


\begin{align*}
    u(x) &= A x + B + e^x \\
    u_x(1) &= A + e^1 = -5 \Rightarrow A = -5 - e \\
    u(-1) &= (5 + e) + B + e^{-1} = 3 \Rightarrow B = 3 - 5 - e - e^{-1} = -(2 + e + e^{-1}) \\
    ~\\
    u(x) &= -(5 + e) x -(2 + e + e^{-1}) + e^{x}
\end{align*}

#### Implementació


Recorda que podem resoldre un sistema 
$$
  A\cdot U = b
$$

Mitjançant:


```
U = numpy.linalg.solve(A, b)
```



In [0]:
?numpy.linalg.solve

In [0]:
# Escriu el codi per implementar el sistema obtingut a dalt


In [0]:
#@title
# Problem setup
a = -1.0
b = 1.0
u_a = 3.0
u_x_b = -5.0
f = lambda x: numpy.exp(x)
u_true = lambda x: -(5.0 + numpy.exp(1.0)) * x - (2.0 + numpy.exp(1.0) + numpy.exp(-1.0)) + numpy.exp(x)

# Discretization
N = 10
x_bc = numpy.linspace(a, b, N + 2)
x = x_bc[1:-1]
delta_x = (b - a) / (N + 1)

# Construct matrix A
A = numpy.zeros((N + 2, N + 2))
diagonal = numpy.ones(N + 2) / delta_x**2
A += numpy.diag(diagonal * -2.0, 0)
A += numpy.diag(diagonal[:-1], 1)
A += numpy.diag(diagonal[:-1], -1)

# Construct RHS
b = f(x_bc)

# Boundary conditions
A[0, 0] = 1.0
A[0, 1] = 0.0
A[-1, -1] = 3.0 / (2.0 * delta_x)
A[-1, -2] = -4.0 / (2.0 * delta_x)
A[-1, -3] = 1.0 / (2.0 * delta_x)

b[0] = u_a
b[-1] = u_x_b

# Solve system
U = numpy.empty(N + 2)
U = numpy.linalg.solve(A, b)

# Plot result
fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(x_bc, U, 'o', label="Computed")
axes.plot(x_bc, u_true(x_bc), 'k', label="True")
axes.set_title("Solution to $u_{xx} = e^x$")
axes.set_xlabel("x")
axes.set_ylabel("u(x)")
plt.show()